### Objective
* Create a database of hexagons with a column for each need polygon

### Workflow
* Import data into spatial dataframes
    * Hexagons feature class
    * Conservation need feature class
* Iterate through each feature in the Needs dataframe
    * Extract the feature's shape
    * Select all Hexagons that have their center in the shape
    * In a new column in the hexagon's dataframe, set values to 1 for selected row

In [1]:
#import packages
from arcgis import GIS
from arcgis.geometry import Point
import pandas as pd

In [24]:
#set input data filenames
fname_need = '../Data/raw/Basic_Data/1_Conservation_Need.shp'
fname_hex = '../Data/raw/Hexagon_files/100km2_D1_LamAzimNeed.shp'

#set output data filenames
fname_hex_new = '../scratch/need_hexagons.shp'

In [6]:
#read data into spatial dataframes
sdf_need = pd.DataFrame.spatial.from_featureclass(fname_need)
sdf_hex = pd.DataFrame.spatial.from_featureclass(fname_hex)

#### Proof of concept
Before iterating through all features, let's examine just one. We want to extract the first shape from the Needs geodataframe, then use that to select hexagons.

More info:
* https://developers.arcgis.com/python/guide/spatially-enabled-dataframe-advanced-topics/

In [7]:
#Get the first needs shape
the_poly = sdf_need.at[0,'SHAPE']

In [8]:
the_hex = sdf_hex.at[0,'SHAPE']
x,y = the_hex.centroid

In [11]:
def get_centroid(theShape):
    x,y = theShape.centroid
    sr = theShape.spatialReference['wkid']
    return Point(F'{{"x": {x}, "y" :{y}, "spatialReference" : {{"wkid" : {sr}}}}}')

In [12]:
#Create a column of centroids - takes a moment
sdf_hex['centroid'] = sdf_hex['SHAPE'].apply(get_centroid)

In [19]:
#Apply the function to all hexagons - takes a moment
sdf_hex['need1']=sdf_hex['centroid'].apply(lambda x: 1 if x.within(the_poly) else 0)
sdf_hex.head()

In [23]:
#Save as feature class
sdf_hex.loc[:,['FID','SHAPE','need1']].spatial.to_featureclass(fname_hex_new)

'C:\\Workspace\\ENV859_2020_prep\\Projects\\MarineProtectedAreas\\AntarcticMPAs\\Scratch\\need_hexagons.shp'

#### The real deal

In [25]:
#re-read data into spatial dataframes
sdf_need = pd.DataFrame.spatial.from_featureclass(fname_need)
sdf_hex = pd.DataFrame.spatial.from_featureclass(fname_hex)

In [26]:
#re-add centroids to the imported spatial dataframe
the_centroids = sdf_hex['SHAPE'].apply(get_centroid)

In [27]:
#Iterate through all features in the needs feature class
for i,row in sdf_need.iterrows():
    #Status
    print(i,end=' ')
    #Get the shape
    the_poly = row['SHAPE']
    #Add new columns in the hex dataframe with 1's and 0's
    col_name=F'need{i+1}'
    sdf_hex[col_name]=the_centroids.apply(lambda x: 1 if x.within(the_poly) else 0)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 

In [28]:
#Tally sum of all added columns
sdf_hex['TOTAL'] = totals = sdf_hex.iloc[:,-i-1:].sum(axis=1)

In [29]:
#Save to new file
outFC = sdf_hex.spatial.to_featureclass(fname_hex_new)